In [213]:
# Import Dependencies
import pandas as pd
from pathlib import Path
import numpy as np

In [214]:
csv_path_flow_2023 = Path("user_flow_fy2023.csv")
csv_path_dist_flow = Path("6. district_flow_estimate.csv")
csv_path_verif_flow = Path("7. verification_flow.csv")
csv_path_suburb_flow = Path("8.PI_Suburban_Flow.csv")
dist_flow = pd.read_csv(csv_path_dist_flow, encoding="UTF-8")
flow_2023 = pd.read_csv(csv_path_flow_2023, encoding="UTF-8")                         
VerificationFlow_2023 = pd.read_csv(csv_path_verif_flow, encoding="UTF-8")
VerificationFlow_2023.round(2)

,station,report_year,annual_flow
0,Main,2023,47.98
1,O Street,2023,4.16
2,Eastside,2023,5.40
3,Poplar Point,2023,7.76
4,Potomac,2023,111.65
5,Rock Creek,2023,3.22
6,Upper Anacostia,2023,1.20
7,Earl Place,2023,0.01


In [215]:
distric_flow_est =  pd.read_csv(csv_path_dist_flow, encoding="UTF-8")
distric_flow_est.head()

,user,flow_route,Flow_length,flow_route_name,sewersheds,pop,emp,visitor,area_acres,infilt_InchMile,...,pro_infiltration_rate,storm_mgd,sanitary_mgd,high_user_mgd,ground_water_pumpage,total_annual_flow,billing_dc_flow,impervious__acres,pervious_acres,pipe_surface_area_acres
0,dc,1,10000.0,little falls,MD-LF,330,874,209,43,0.00,...,NaN,NaN,0.09,NaN,0.02,NaN,NaN,NaN,NaN,0.67
1,dc,1,10000.0,little falls,LF Boundary (BN-68),10084,8478,0,870,781.85,...,NaN,NaN,1.42,0.16,0.04,NaN,NaN,NaN,NaN,10.86
2,dc,1,10000.0,little falls,UPI-NM1 (BN-68),259,431,0,48,13.03,...,NaN,NaN,0.04,0.49,NaN,NaN,NaN,NaN,NaN,0.36
3,dc,1,10000.0,little falls,UPI-NM2 (BN-69),2241,1693,0,202,151.42,...,NaN,NaN,0.31,NaN,NaN,NaN,NaN,NaN,NaN,1.91
4,dc,1,10000.0,little falls,UPI-NM3 (M-4317),585,440,0,107,108.90,...,NaN,NaN,0.08,0.24,NaN,NaN,NaN,NaN,NaN,1.95


# Inputs

In [216]:
rainfall = 42
#1 ac-ft/min = 
gal_per_min = 325851.428571
Drainage_coeff_for_imperv_area = 0.9
#ratio of runoff to rainfall (Design Manual, Vol. 3, p. 3-D1-7), in./in.
Drainage_Coeff_for_perv_Area = 0.3
Wastewater_per_capita =110
# mgd (From Section 3.5.2 Unit Wastwater Flow Factor)
Theoretical_Qlf = 16.6
Factor_for_High_Water_Use= Theoretical_Qlf/19.05
# Infiltration (Table 6-1)= ((rainfall-40.84)*((50.32-40.84)/(52-64)))+52
Infiltration = (((rainfall-40.84)*((50.32-40.84)/(52-64)))+52)
Factor_infil= Infiltration/distric_flow_est["infil_MGD"].sum()
#  gallons/day from Tech Memo 18, p. 3-14
# Rainfall 34.90 inches	Comparing to Table 6-1 'Projected District Wastewater Quantities'
# Rainfall ft = 2.908333 ac-ft/min 325,851 gal/min

In [217]:
print("Rain Fall 2023                 =",rainfall)
print("Infiltration based on rainfall = ",format(Infiltration, '.2f'))
print("Infiltration Factor            = ",format(Factor_infil, '.2f') )
print("Drainage_coeff_for_imperv_area =",Drainage_coeff_for_imperv_area)
print("Drainage_Coeff_for_perv_Area   = ", Drainage_Coeff_for_perv_Area)
print("Wastewater_per_capita          = ",Wastewater_per_capita)
print("Theoretical_Qlf                = ",Theoretical_Qlf)
print("Factor for High Water Use      = ", format(Factor_for_High_Water_Use, '.2f'))

Rain Fall 2023                 = 42
Infiltration based on rainfall =  51.08
Infiltration Factor            =  0.48
Drainage_coeff_for_imperv_area = 0.9
Drainage_Coeff_for_perv_Area   =  0.3
Wastewater_per_capita          =  110
Theoretical_Qlf                =  16.6
Factor for High Water Use      =  0.87


Calculate District Sewershed Flows incl Sanitary, Storm, High Water Users and Goundwater, finally Total district flow

In [218]:
distric_flow_est['pro_infiltration_rate'] = distric_flow_est["infil_MGD"]*Factor_infil
distric_flow_est['storm_mgd'] = (distric_flow_est["impervious__acres"]*Drainage_coeff_for_imperv_area*(rainfall/12)*gal_per_min
                                + 
                                distric_flow_est["pervious_acres"]*Drainage_Coeff_for_perv_Area* (rainfall/12)*gal_per_min)/365.242199/1000000
distric_flow_est['storm_mgd'] =distric_flow_est['storm_mgd'] .fillna(0)

distric_flow_est['sanitary_mgd'] = ((distric_flow_est["pop"]+ distric_flow_est["emp"]/3+ 
                                    distric_flow_est["visitor"])*Wastewater_per_capita )/1000000
distric_flow_est['high_user_mgd'] = distric_flow_est['high_user_mgd'].fillna(0)
distric_flow_est['ground_water_pumpage'] = distric_flow_est['ground_water_pumpage'].fillna(0)
# distric_flow_est['DataFrame Column'] = distric_flow_est['DataFrame Column'].fillna(0)
                 
distric_flow_est['total_annual_flow'] = (distric_flow_est["storm_mgd"]+ distric_flow_est["sanitary_mgd"] +\
                                    distric_flow_est["high_user_mgd"]+distric_flow_est["ground_water_pumpage"] +\
                                    distric_flow_est["pro_infiltration_rate"]).round(2)

district_flow = distric_flow_est[["flow_route","sewersheds","storm_mgd","sanitary_mgd","pro_infiltration_rate",
                  "high_user_mgd","ground_water_pumpage","total_annual_flow","Flow_length","user"]]

district_flow = district_flow[["flow_route","sewersheds","total_annual_flow","Flow_length","user"]]
DC_est_annualflow = district_flow["total_annual_flow"].sum()
print("Estimated District Flow = ",DC_est_annualflow,"mgd")
district_flow.head()

Estimated District Flow =  190.43 mgd


,flow_route,sewersheds,total_annual_flow,Flow_length,user
0,1,MD-LF,0.11,10000.0,dc
1,1,LF Boundary (BN-68),2.60,10000.0,dc
2,1,UPI-NM1 (BN-68),0.55,10000.0,dc
3,1,UPI-NM2 (BN-69),0.50,10000.0,dc
4,1,UPI-NM3 (M-4317),0.46,10000.0,dc


In [219]:
print("Storm, mgd = ",distric_flow_est['storm_mgd'].sum().round(2))
print("Sanitary, mgd = ",distric_flow_est['sanitary_mgd'].sum().round(2))
print("Infiltration,mgd = ",distric_flow_est['pro_infiltration_rate'].sum().round(2))
print("TotalAnnualFlow, mgd = ",distric_flow_est['total_annual_flow'].sum().round(2))

Storm, mgd =  23.14
Sanitary, mgd =  89.05
Infiltration,mgd =  51.08
TotalAnnualFlow, mgd =  190.43


In [220]:
district_flow_route8 = district_flow[
                                    (district_flow["flow_route"] == 1) 
                                   |
                                    (district_flow["flow_route"] == 2 )
                                    ]
district_flow_route8.rename(columns={'sewersheds': 'Input_Name'}, inplace=True)
district_flow_route8.rename(columns={'total_annual_flow': 'Flow'}, inplace=True)

district_flow_route8

C:\Users\eyasu\AppData\Local\Temp\ipykernel_29164\1447077786.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  district_flow_route8.rename(columns={'sewersheds': 'Input_Name'}, inplace=True)
C:\Users\eyasu\AppData\Local\Temp\ipykernel_29164\1447077786.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  district_flow_route8.rename(columns={'total_annual_flow': 'Flow'}, inplace=True)


,flow_route,Input_Name,Flow,Flow_length,user
0,1,MD-LF,0.11,10000.0,dc
1,1,LF Boundary (BN-68),2.60,10000.0,dc
2,1,UPI-NM1 (BN-68),0.55,10000.0,dc
3,1,UPI-NM2 (BN-69),0.50,10000.0,dc
4,1,UPI-NM3 (M-4317),0.46,10000.0,dc
5,2,UPI-7,0.53,5000.0,dc
6,2,UPI-NM4,0.53,5000.0,dc
7,2,UPI-1,4.21,5000.0,dc
8,2,CS0 029 (College Pond),2.71,5000.0,dc
9,2,CSO 057/058,1.72,5000.0,dc


# User Flow Reported Fiscal Year 2023

In [221]:
district_flow_route8['Route'] =[8,8,8,8,8,8,8,8,8,8,8,8,8]
Route_8_Flow = district_flow_route8[["user","Route","Input_Name","Flow","Flow_length"]]
Route_8_Flow

C:\Users\eyasu\AppData\Local\Temp\ipykernel_29164\3279403454.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  district_flow_route8['Route'] =[8,8,8,8,8,8,8,8,8,8,8,8,8]


,user,Route,Input_Name,Flow,Flow_length
0,dc,8,MD-LF,0.11,10000.0
1,dc,8,LF Boundary (BN-68),2.60,10000.0
2,dc,8,UPI-NM1 (BN-68),0.55,10000.0
3,dc,8,UPI-NM2 (BN-69),0.50,10000.0
4,dc,8,UPI-NM3 (M-4317),0.46,10000.0
5,dc,8,UPI-7,0.53,5000.0
6,dc,8,UPI-NM4,0.53,5000.0
7,dc,8,UPI-1,4.21,5000.0
8,dc,8,CS0 029 (College Pond),2.71,5000.0
9,dc,8,CSO 057/058,1.72,5000.0


Flow Share calculation  Inside the District Boundary

# WSSC Billing Flow Report for Fiscal Year 2023

In [222]:
flow_2023_wssc = flow_2023[flow_2023["user_name"]=='wssc']
flow_2023_wssc_flow = flow_2023_wssc.groupby(["input_name"])[["monthly_flow"]].sum()/365.25
print("WSSC Billing Total Flow = ",flow_2023_wssc_flow.sum().map("{:,.2f}".format))
flow_2023_wssc_flow.round(2).head()

WSSC Billing Total Flow =  monthly_flow    97.16
dtype: object


,monthly_flow
input_name,
BARNABY,0.76
BELT,0.16
CABIN_JOHN,5.68
DC-WSSC - ANACOSTIA,52.12
EAST BEACH DRIVE,0.70


# Fairfax Billing Flow Report for Fiscal Year 2023

In [223]:
flow_2023_fairfax = flow_2023[flow_2023["user_name"]=='fairfax']
flow_2023_fairfax_flow = flow_2023_fairfax.groupby(["input_name"])[["monthly_flow"]].sum()/365.25
print("Fairfax Billing Total Flow = ",flow_2023_fairfax_flow.sum().map("{:,.2f}".format))
flow_2023_fairfax_flow.round(2)

Fairfax Billing Total Flow =  monthly_flow    16.83
dtype: object


,monthly_flow
input_name,
AT & T - B3046 - P.I.,6.26
CHAIN BRIDGE - G4102 - DOWN,1.88
ROCK HILL ROAD - A3003 - P.I.,0.57
SUGARLAND RUN - B2006 - P.I.,3.51
Sully_Rd_1,2.69
Sully_Rd_2,1.88
TREESIDE - B5706 - P.I.,0.01
UNMETERED,0.02


# Loudoun Billing Flow Report for Fiscal Year 2023

In [224]:
flow_2023_Loudoun = flow_2023[flow_2023["user_name"]=='loudoun']
flow_2023_Loudoun_flow = flow_2023_Loudoun.groupby(["input_name"])[["monthly_flow"]].sum()/365.25
print("Loudoun Billing Total Flow = ",flow_2023_Loudoun_flow.sum().map("{:,.2f}".format),"mgd")
flow_2023_Loudoun_flow.round(2).head()

Loudoun Billing Total Flow =  monthly_flow    6.59
dtype: object mgd


,monthly_flow
input_name,
BEAUMEADE #1,0.22
BEAUMEADE 2 SM,0.02
BOISE CASCADE,0.37
BRIDGES 625/604 - S8,0.45
BROAD RUN,0.31


# Vienna Billing Flow Report for Fiscal Year 2023

In [225]:
flow_2023_Vienna = flow_2023[flow_2023["user_name"]=='vienna']
flow_2023_Vienna_flow = flow_2023_Vienna.groupby(["input_name"])[["monthly_flow"]].sum()/365.25
print("Town of Vienna Billing Total Flow = ",flow_2023_Vienna_flow.sum().map("{:,.2f}".format))
flow_2023_Vienna_flow.round(2)

Town of Vienna Billing Total Flow =  monthly_flow    1.06
dtype: object


,monthly_flow
input_name,
CREEK CROSSING,0.43
NORTHSIDE,0.53
UNMETERED - VIENNA,0.11


# Dulles International Airport Billing Flow Report for Fiscal Year 2023

In [226]:
flow_2023_Dulles = flow_2023[flow_2023["user_name"]=='dulles']
flow_2023_Dulles_flow = flow_2023_Dulles.groupby(["input_name"])[["monthly_flow"]].sum()/365.25
print("Dulles Billing Total Flow = ",flow_2023_Dulles_flow.sum().map("{:,.2f}".format))
flow_2023_Dulles_flow.round(2)

Dulles Billing Total Flow =  monthly_flow    0.81
dtype: object


,monthly_flow
input_name,
DULLES - S -110,0.77
DULLES - S -14,0.04


In [227]:
PI_Suburban_Flow = pd.read_csv(csv_path_suburb_flow, encoding="UTF-8")
PI_Suburban_Flow= PI_Suburban_Flow[["user","Route","Input_Name","Flow","Flow_length"]]
PI_Suburban_Flow.round(2).head()

,user,Route,Input_Name,Flow,Flow_length
0,Fairfax,1,Sully Road #1,2.69,450
1,Fairfax,1,Sully Road #2,1.88,5981
2,Fairfax,1,Rock Hill Rd,0.57,9519
3,Dulles,2,Dulles,0.81,1700
4,Loudoun County,2,Indian Creek,0.68,4796


In [228]:
# Route 1 Dulles Aiport Area
Route_1_Flow = PI_Suburban_Flow[(PI_Suburban_Flow["Route"] == 1) ]
Route_1_Flow['flow_X_length'] = ""
Route_1_Flow['Share'] = ""
for rows in Route_1_Flow:
    Route_1_Flow["flow_X_length"] = Route_1_Flow["Flow"]*Route_1_Flow["Flow_length"]
sum_length =   Route_1_Flow["flow_X_length"].sum() 
for rows in Route_1_Flow :
     Route_1_Flow["Share"] = (Route_1_Flow["flow_X_length"]/sum_length)*100
Route_1_Flow.round(2)
user_share = Route_1_Flow.groupby(["user"]).sum()["Share"]
user_share
user_share = pd.DataFrame(user_share)
user_share.astype(str).add('%')

C:\Users\eyasu\AppData\Local\Temp\ipykernel_29164\667585445.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  Route_1_Flow['flow_X_length'] = ""
C:\Users\eyasu\AppData\Local\Temp\ipykernel_29164\667585445.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  Route_1_Flow['Share'] = ""
C:\Users\eyasu\AppData\Local\Temp\ipykernel_29164\667585445.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the cav

,Share
user,
Fairfax,100.0%


In [229]:
Route_1_Flow_transfer =   Route_1_Flow.groupby(["user"]).sum() 
Route_1_Flow_transfer = Route_1_Flow_transfer [["Flow"]]
Route_1_Flow_transfer = Route_1_Flow_transfer.reset_index()
Route_1_Flow_transfer

,user,Flow
0,Fairfax,5.14


In [230]:
# Route 2 Loudoun County
Route_2_Flow = PI_Suburban_Flow[(PI_Suburban_Flow["Route"] == 2) ]
Route_2_Flow['flow_X_length'] = ""
Route_2_Flow['Share'] = ""

Route_2_Flow = pd.concat([Route_2_Flow, Route_1_Flow_transfer], ignore_index=True)

# Route_2_Flow = Route_2_Flow.groupby(["user"])
Route_2_Flow.head()

C:\Users\eyasu\AppData\Local\Temp\ipykernel_29164\743048638.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  Route_2_Flow['flow_X_length'] = ""
C:\Users\eyasu\AppData\Local\Temp\ipykernel_29164\743048638.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  Route_2_Flow['Share'] = ""


,user,Route,Input_Name,Flow,Flow_length,flow_X_length,Share
0,Dulles,2.0,Dulles,0.81,1700.0,,
1,Loudoun County,2.0,Indian Creek,0.68,4796.0,,
2,Loudoun County,2.0,Broad Run,0.12,4796.0,,
3,Loudoun County,2.0,Mercure,0.02,4796.0,,
4,Loudoun County,2.0,Loudoun Station 2,0.02,4796.0,,


In [231]:
for rows in Route_2_Flow:
    Route_2_Flow["flow_X_length"] = Route_2_Flow["Flow"]*Route_2_Flow["Flow_length"]
Route_2_Flow    
flow_length_sum = Route_2_Flow["flow_X_length"].sum()
flow_length_sum 
flow_user_sum = (Route_2_Flow.groupby(["user"]).sum()["flow_X_length"]/flow_length_sum)*100
flow_user_sum = pd.DataFrame(flow_user_sum )
flow_user_sum

,flow_X_length
user,
Dulles,2.300665
Fairfax,0.000000
Loudoun County,97.699335


In [232]:
Route_2_Flow_transfer =   Route_2_Flow.groupby(["user"]).sum() 
Route_2_Flow_transfer = Route_2_Flow_transfer [["Flow"]]
Route_2_Flow_transfer = Route_2_Flow_transfer.reset_index()
Route_2_Flow_transfer["Flow_length"] ="9000"
Route_2_Flow_transfer 

,user,Flow,Flow_length
0,Dulles,0.81,9000
1,Fairfax,8.76,9000
2,Loudoun County,12.88,9000


In [233]:
#Route 3 Sugarland Run

In [234]:
#Route 4 Greafalls_VA
Route_4_Flow = PI_Suburban_Flow[(PI_Suburban_Flow["Route"] == 4) ]
Route_4_Flow
Route_4_Flow = pd.concat([Route_4_Flow , Route_2_Flow_transfer ], ignore_index=True)
Route_4_Flow['Flow_X_length'] = ""
Route_4_Flow['%_Share'] = ""
Route_4_Flow.Flow_length  = (Route_4_Flow.Flow_length ).astype("float")
for rows in Route_4_Flow:
    Route_4_Flow["Flow_X_length"] = Route_4_Flow["Flow"]*Route_4_Flow["Flow_length"]
Route_4_Flow

,user,Route,Input_Name,Flow,Flow_length,Flow_X_length,%_Share
0,Loudoun County,4.0,Northwestern,0.18,3995.0,719.10,
1,Loudoun County,4.0,Northeastern,0.08,18955.0,1516.40,
2,Fairfax,4.0,NICHOLS RUN,0.02,18955.0,379.10,
3,WSSC,4.0,Watts_River_PI Branch,5.22,14057.0,73377.54,
4,WSSC,4.0,Muddy_Branch_PI,6.77,14057.0,95165.89,
5,Vienna,4.0,Town of Vienna,1.06,14057.0,14900.42,
6,NPS,4.0,Nationan Park Service,0.03,14057.0,421.71,
7,Fairfax,4.0,Great Falls,9.31,21543.0,200565.33,
8,Dulles,NaN,NaN,0.81,9000.0,7290.00,
9,Fairfax,NaN,NaN,8.76,9000.0,78840.00,


In [235]:
Flow_length_sum = Route_4_Flow["Flow_X_length"].sum()
Flow_length_sum
Route_4_Flow = Route_4_Flow.groupby(["user"]).sum(["Flow_X_length"])
Route_4_Flow = pd.DataFrame(Route_4_Flow)
Route_4_Flow 
Route_4_Flow ["%_Share"] = (Route_4_Flow.groupby(["user"]).sum()["Flow_X_length"]/Flow_length_sum)*100

# flow_user_sum = pd.DataFrame(flow_user_sum )
Route_4_Flow.round(2)

,Route,Flow,Flow_length,Flow_X_length,%_Share
user,,,,,
Dulles,0.0,0.81,9000.0,7290.00,1.24
Fairfax,8.0,18.09,49498.0,279784.43,47.49
Loudoun County,8.0,13.14,31950.0,118155.50,20.06
NPS,4.0,0.03,14057.0,421.71,0.07
Vienna,4.0,1.06,14057.0,14900.42,2.53
WSSC,8.0,11.99,28114.0,168543.43,28.61


In [236]:
Route_4_Flow_transfer =   Route_4_Flow.groupby(["user"]).sum() 
Route_4_Flow_transfer = Route_4_Flow_transfer [["Flow"]]
Route_4_Flow_transfer = Route_4_Flow_transfer.reset_index()
Route_4_Flow_transfer["Flow_length"] ="9000"
Route_4_Flow_transfer

,user,Flow,Flow_length
0,Dulles,0.81,9000
1,Fairfax,18.09,9000
2,Loudoun County,13.14,9000
3,NPS,0.03,9000
4,Vienna,1.06,9000
5,WSSC,11.99,9000


In [237]:
#Route 5 Montgomery_MD

In [238]:
#Route 6 Difficult Run

In [239]:
#Route 7 Clara Barton pkwy
Route_7_Flow = PI_Suburban_Flow[(PI_Suburban_Flow["Route"] == 6) ]
Route_7_Flow

,user,Route,Input_Name,Flow,Flow_length
42,Fairfax,6,Scott Run,2.79,4000
43,Navy,6,Naval Ship R&D,0.07,1437
44,WSSC,6,Rock_Run_PI,0.74,1437
45,WSSC,6,WSSC_unmetered_PI,0.10,12155
46,WSSC,6,Cabin_PI,5.68,13251


In [240]:
#Route 7 Clara Barton pkwy
Route_7_Flow = PI_Suburban_Flow[(PI_Suburban_Flow["Route"] == 4) ]
Route_7_Flow
Route_7_Flow = pd.concat([Route_7_Flow , Route_4_Flow_transfer ], ignore_index=True)
Route_7_Flow['Flow_X_length'] = ""
Route_7_Flow['%_Share'] = ""
Route_7_Flow.Flow_length  = (Route_7_Flow.Flow_length ).astype("float")
for rows in Route_4_Flow:
    Route_7_Flow["Flow_X_length"] = Route_7_Flow["Flow"]*Route_7_Flow["Flow_length"]
Route_7_Flow


,user,Route,Input_Name,Flow,Flow_length,Flow_X_length,%_Share
0,Loudoun County,4.0,Northwestern,0.18,3995.0,719.10,
1,Loudoun County,4.0,Northeastern,0.08,18955.0,1516.40,
2,Fairfax,4.0,NICHOLS RUN,0.02,18955.0,379.10,
3,WSSC,4.0,Watts_River_PI Branch,5.22,14057.0,73377.54,
4,WSSC,4.0,Muddy_Branch_PI,6.77,14057.0,95165.89,
5,Vienna,4.0,Town of Vienna,1.06,14057.0,14900.42,
6,NPS,4.0,Nationan Park Service,0.03,14057.0,421.71,
7,Fairfax,4.0,Great Falls,9.31,21543.0,200565.33,
8,Dulles,NaN,NaN,0.81,9000.0,7290.00,
9,Fairfax,NaN,NaN,18.09,9000.0,162810.00,


In [241]:
Flow_length_sum = Route_7_Flow["Flow_X_length"].sum()
Flow_length_sum
Route_7_Flow = Route_4_Flow.groupby(["user"]).sum(["Flow_X_length"])
Route_7_Flow = pd.DataFrame(Route_7_Flow)
Route_7_Flow 
Route_7_Flow ["%_Share"] = (Route_7_Flow.groupby(["user"]).sum()["Flow_X_length"]/Flow_length_sum)*100

# flow_user_sum = pd.DataFrame(flow_user_sum )
Route_7_Flow.round(2)

,Route,Flow,Flow_length,Flow_X_length,%_Share
user,,,,,
Dulles,0.0,0.81,9000.0,7290.00,0.92
Fairfax,8.0,18.09,49498.0,279784.43,35.28
Loudoun County,8.0,13.14,31950.0,118155.50,14.90
NPS,4.0,0.03,14057.0,421.71,0.05
Vienna,4.0,1.06,14057.0,14900.42,1.88
WSSC,8.0,11.99,28114.0,168543.43,21.25


In [242]:
Route_7_Flow_transfer =   Route_7_Flow.groupby(["user"]).sum() 
Route_7_Flow_transfer = Route_7_Flow_transfer [["Flow"]]
Route_7_Flow_transfer = Route_7_Flow_transfer.reset_index()
Route_7_Flow_transfer["Flow_length"] ="9000"
Route_7_Flow_transfer

,user,Flow,Flow_length
0,Dulles,0.81,9000
1,Fairfax,18.09,9000
2,Loudoun County,13.14,9000
3,NPS,0.03,9000
4,Vienna,1.06,9000
5,WSSC,11.99,9000


In [243]:
Route_8_Flow

,user,Route,Input_Name,Flow,Flow_length
0,dc,8,MD-LF,0.11,10000.0
1,dc,8,LF Boundary (BN-68),2.60,10000.0
2,dc,8,UPI-NM1 (BN-68),0.55,10000.0
3,dc,8,UPI-NM2 (BN-69),0.50,10000.0
4,dc,8,UPI-NM3 (M-4317),0.46,10000.0
5,dc,8,UPI-7,0.53,5000.0
6,dc,8,UPI-NM4,0.53,5000.0
7,dc,8,UPI-1,4.21,5000.0
8,dc,8,CS0 029 (College Pond),2.71,5000.0
9,dc,8,CSO 057/058,1.72,5000.0


In [264]:
#Route 8 DC
Route_8_Flow
Route_8_Flow['Flow_X_length'] = ""

Flow_length_sum = Route_8_Flow["Flow_X_length"].sum()
Route_8_Flow = pd.concat([Route_8_Flow , Route_7_Flow_transfer ], ignore_index=True)
Route_8_Flow 
Route_8_Flow.Flow_length  = (Route_8_Flow.Flow_length ).astype("float")
for rows in Route_4_Flow:
    Route_8_Flow["Flow_X_length"] = Route_8_Flow["Flow"]*Route_8_Flow["Flow_length"]
Route_8_Flow 




TypeError: 'DataFrameGroupBy' object does not support item assignment

In [263]:
Route_8_Flow = Route_8_Flow.groupby(["user"])

# flow_user_sum = pd.DataFrame(flow_user_sum )
Route_8_Flow

AttributeError: 'DataFrameGroupBy' object has no attribute 'groupby'